In [ ]:
import rasterio
import shapely
from shapely.geometry import (Polygon, shape)
from rasterio.mask import mask
from pyproj import Proj, transform
import geopandas as gpd
import os

In [ ]:
materials = ["concrete_cement","healthy_metal","incomplete","irregular_metal","other"]
for name in materials:
    os.makedirs('Processed data/train/'+name)
    
os.makedirs('Processed data/test')

In [ ]:
for country in ["colombia", "guatemala", "st_lucia"]:
    regions = {"st_lucia":["dennery"], "colombia":["borde_rural","borde_soacha"], "guatemala":["mixco_1_and_ebenezer","mixco_3"]}
    for region in regions[country]:
        src = rasterio.open('/home/cupgreek/Documents/stac/'+country+'/'+region+'/'+region+'_ortho-cog.tif')
        print(region+':'+str(src.crs))

In [ ]:
#Function to extract the rooftop image from the source image and save it in '.tif' format
epsg = {'colombia':'32618', 'guatemala':'32616', 'st_lucia':'32620'}
for country in ['colombia','guatemala','st_lucia']:
    regions = {'st_lucia':['dennery'], 'colombia':['borde_rural','borde_soacha'], 'guatemala':['mixco_1_and_ebenezer','mixco_3']}
    for region in regions[country]:
        src = rasterio.open('/home/cupgreek/Documents/stac/'+country+'/'+region+'/'+region+'_ortho-cog.tif')
        for case in ['train', 'test']:
            aoi_geojson = gpd.read_file('/home/cupgreek/Documents/stac/'+country+'/'+region+'/'+case+'-'+region+'.geojson')
            for i in range(aoi_geojson.shape[0]):
                cord = aoi_geojson['geometry'][i].bounds
                inProj = Proj(init = 'epsg:4326')
                print(country,epsg[country],'epsg:'+epsg[country])
                outProj = Proj(init = 'epsg:'+epsg[country])
                (x1,y1,x2,y2) = cord
                x1_new,y1_new = transform(inProj,outProj,x1,y1)
                x2_new,y2_new = transform(inProj,outProj,x2,y2)
                ar_new = shapely.geometry.box(x1_new, y1_new, x2_new, y2_new, ccw=True)
                crop, cropTransform = mask(src, [ar_new], crop=True)
                meta = src.meta.copy()
                meta.update({'transform':cropTransform, 'height':crop.shape[1], 'width':crop.shape[2]})
                if case == 'train':
                    with rasterio.open('Processed data/'+case+'/'+aoi_geojson['roof_material'][i]+'/'+aoi_geojson['id'][i]+'.tif', 'w', **meta) as dst:
                        dst.write(crop)
                if case == 'test':
                    with rasterio.open('Processed data/'+case+'/'+aoi_geojson['id'][i]+'.tif', 'w', **meta) as dst:
                        dst.write(crop)